In [3]:
# Run to stitch the webscraped NWS data together (collected 3 days at a time).  
# input: ../data/nws_past_3/[code for report day and time].csv  
# output: ../data/nws_weather_data.csv

In [5]:
import pandas as pd
import os
import re

In [ ]:
# step 1 - merge!

# step 1a - loop through the files, merging them

In [160]:
nws = pd.DataFrame(columns = ['year','month','day_of_month','time',
                             'wind','temp','precip_1_h','precip_3_h','precip_6_h'])

In [162]:
folder = '../data/nws_past_3/'
files = [f for f in os.listdir(folder)]

print('starting...')

# processing file X of Y: filename.csv

x = 1
y = len(files)

for file in files :
    print(f"\rprocessing file {x} of {y}: {file}                  ", end="")
    next_df = pd.read_csv(folder + str(file))
    nws = pd.concat([nws, next_df])
    x += 1

print()
print('done!')

starting...
processing file 30 of 30: 2025-04-11T18-14-25+00-00.csv                  
done!


C:\Users\Garrett\AppData\Local\Temp\ipykernel_23856\677830200.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  nws = pd.concat([nws, next_df])


In [164]:
nws = nws.reset_index(drop=True)

In [166]:
# some of the times have leading zeroes, some do not.  Strip the first leading zero.
for index, row in nws.iterrows() :
    corrected_time = re.sub(r'^0', '', row['time'])
    # and to make sure it still shows up as 0:53 for midnight...
    corrected_time = re.sub(r'^:', '0:', corrected_time)
    nws.loc[index,'time'] = corrected_time

In [168]:
nws = nws.drop_duplicates().reset_index(drop=True)

In [ ]:
# Next step... fill in the missing hours data (every once in a while, it seems to not record stats for 1 hour every 2 days or so.)
# Decide how to handle...
# And continue processing the combined file!  Do I need any other conversions / modifications?

In [170]:
nws = nws.sort_values(by = ['year','month','day_of_month','time'], ascending = False)
nws

,year,month,day_of_month,time,wind,temp,precip_1_h,precip_3_h,precip_6_h
317,2025,4,11,9:53,N 7,53.1,NaN,NaN,NaN
318,2025,4,11,8:53,NW 9,52.0,NaN,NaN,NaN
319,2025,4,11,7:53,NW 8,50.0,NaN,NaN,NaN
320,2025,4,11,6:53,NW 9,50.0,NaN,NaN,NaN
321,2025,4,11,5:53,W 7,48.9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
57,2025,3,26,14:53,NaN,66.9,NaN,NaN,NaN
58,2025,3,26,13:53,NaN,66.9,NaN,NaN,NaN
59,2025,3,26,12:53,NaN,64.9,NaN,NaN,NaN
60,2025,3,26,11:53,NaN,61.0,NaN,NaN,NaN


In [172]:
# eventually, save it:
nws.to_csv('../data/nws_weather_data.csv', index=False)

In [158]:
# 2025	4	8	temp 43
nws[nws['temp']==43]

,year,month,day_of_month,time,wind,temp,precip_1_h,precip_3_h,precip_6_h
314,2025,4,8,9:53,NaN,43.0,NaN,NaN,NaN
320,2025,4,8,3:53,NaN,43.0,NaN,NaN,NaN
294,2025,4,8,09:53,NaN,43.0,NaN,NaN,NaN
300,2025,4,8,03:53,NaN,43.0,NaN,NaN,NaN


In [118]:
# 2025	4	8	temp 43
nws[nws['temp']==43]

,year,month,day_of_month,time,wind,temp,precip_1_h,precip_3_h,precip_6_h
60,2025,4,8,9:53,NaN,43.0,NaN,NaN,NaN
66,2025,4,8,3:53,NaN,43.0,NaN,NaN,NaN


In [ ]:
# this is copied from the active_dispatch merge, to reference

In [9]:
files = [f for f in os.listdir('../data/active_dispatch/')]

print('starting...')

# processing file X of Y: filename.csv

x = 1
y = len(files)

for file in files :
    print(f"\rprocessing file {x} of {y}: {file}                  ", end="")
    next_df = pd.read_csv('../data/active_dispatch/' + str(file))
    ad = pd.concat([ad, next_df]).drop_duplicates()
    x += 1

ad.reset_index(drop=True)

print()
print('done!')

starting...
processing file 2088 of 2088: 1744387021000.csv                  done!


In [10]:
print(len(ad)) # how many records?

26184


In [11]:
# make sure I know which columns have nulls...
nulls_per_column = ad.isnull().sum()
print(nulls_per_column)

incident_type_code          0
incident_type_name          0
call_received_time          0
location                    0
location_description    26184
city_name                   0
last_updated                0
dtype: int64


In [12]:
# step 1b - group by to get rid of the duplicates, keeping only the latest last_updated record
# Group By [incident_type_code], [incident_type_name], etc... EVERYTHING EXCEPT [last_updated]... use the MAX([last_updated])

In [13]:
#grouped_sum = df.groupby(['Category', 'Subcategory'])['Value'].sum()
#ad.groupby(['incident_type_code','incident_type_name','call_received_time','location',
#                             'location_description','city_name'])['last_updated'].max()
ad = ad.groupby(['incident_type_code','incident_type_name','call_received_time',
                 'location','city_name'])['last_updated'].max().reset_index()

In [14]:
ad = ad.sort_values(by = 'call_received_time').reset_index()

In [15]:
ad

,index,incident_type_code,incident_type_name,call_received_time,location,city_name,last_updated
0,514,70A,RESIDENCE-BURGLARY ALARM,1743085667000,719 MYRTLE ST,EAST,1743103621000
1,1145,71A,NON-RESIDENCE-BURGLARY ALARM,1743085918000,3188 DICKERSON PIKE,EAST,1743103621000
2,2260,83P,SHOTS FIRED,1743093178000,1433 PENNOCK AVE,EAST,1743115680000
3,2223,71P,BURGLARY-NON-RESIDENCE BREAK-IN,1743094882000,360 WALLACE RD,PARAGON MILLS,1743110280000
4,515,70A,RESIDENCE-BURGLARY ALARM,1743095110000,1236 BRENTWOOD HIGHLANDS DR,NIPPERS CORNER,1743103621000
...,...,...,...,...,...,...,...
3012,2221,71A,NON-RESIDENCE-BURGLARY ALARM,1744384055000,2606 EUGENIA AVE,WOODBINE,1744387021000
3013,2222,71A,NON-RESIDENCE-BURGLARY ALARM,1744384891000,3106 BELMONT BLVD,BELMONT,1744385220000
3014,129,53A,ROBERRY/HOLD UP ALARM,1744385291000,800 FORT NEGLEY BLVD,FORT NEGLEY,1744386481000
3015,988,70A,RESIDENCE-BURGLARY ALARM,1744385848000,2927 GLENMEADE DR,INGLEWOOD,1744387021000


### step 2 - calculate and format!

In [28]:
calculated_ad = pd.DataFrame(columns = ['tencode',
                                        'incident_type_name',
                                        'call_received_date_time', # convert from UNIX time stamp
                                        'cr_date',
                                        'cr_time',  #			5:45 PM
                                        'cr_display_month', # Jan, Feb, Mar, etc
                                        'cr_month_num', # 1, 2, 3
                                        'cr_year',
                                        'cr_weekday', # Sun, Mon, Tues, Wed
                                        'cr_weekday_num', # 1, 2, 3, 4
                                        'cr_display_hour', #		0-23
                                        'cr_hour', #			1-12
                                        'cr_min',
                                        'cr_ampm',
                                        'address', # -> location + ", NASHVILLE, TN"
                                        #	(Estimated Resolution Time: ert)
                                        #	add 240,000 to the UNIX time stamp of the last_updated time (4 minutes * 60 seconds * 1000 miliseconds)
                                        'ert_date',
                                        'ert_time',
                                        'ert_display_month',
                                        'ert_month_num',
                                        'ert_year',
                                        'ert_weekday',
                                        'ert_weekday_num',
                                        'ert_display_hour',
                                        'ert_hour',
                                        'ert_min',
                                        'ert_ampm'
                                       ])

In [32]:
# processing row X (index) of Y: row[call_received_time]

x = 1
y = len(ad)

for index, row in ad.iterrows() :
    print(f"\rprocessing row {x} (index {index}) of {y}: call received: {row['call_received_time']}          ", end="")
    code = row['incident_type_code']
    match = re.search(r'^(\d+)', code)    # search for a digit or multiple, followed by a space, followed by any number of any characters
    if match :
        code = match.group(1)
    else :
        print('Missed a regex match in a row... look into this and try again')
    
    new_row = {'tencode': code,
               'incident_type_name': row['incident_type_name']}
    calculated_ad.loc[len(calculated_ad)] = new_row
    x += 1

print()
print('done!')

processing row 3017 (index 3016) of 3017: call received: 1744386630000          
done!


In [36]:
# look for nulls again...
nulls_per_column = calculated_ad.isnull().sum()
print(nulls_per_column)

tencode                       0
incident_type_name            0
call_received_date_time    6034
cr_date                    6034
cr_time                    6034
cr_display_month           6034
cr_month_num               6034
cr_year                    6034
cr_weekday                 6034
cr_weekday_num             6034
cr_display_hour            6034
cr_hour                    6034
cr_min                     6034
cr_ampm                    6034
address                    6034
ert_date                   6034
ert_time                   6034
ert_display_month          6034
ert_month_num              6034
ert_year                   6034
ert_weekday                6034
ert_weekday_num            6034
ert_display_hour           6034
ert_hour                   6034
ert_min                    6034
ert_ampm                   6034
dtype: int64


In [34]:
calculated_ad

,tencode,incident_type_name,call_received_date_time,cr_date,cr_time,cr_display_month,cr_month_num,cr_year,cr_weekday,cr_weekday_num,...,ert_time,ert_display_month,ert_month_num,ert_year,ert_weekday,ert_weekday_num,ert_display_hour,ert_hour,ert_min,ert_ampm
0,70,RESIDENCE-BURGLARY ALARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71,NON-RESIDENCE-BURGLARY ALARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,83,SHOTS FIRED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71,BURGLARY-NON-RESIDENCE BREAK-IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,70,RESIDENCE-BURGLARY ALARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6029,71,NON-RESIDENCE-BURGLARY ALARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6030,71,NON-RESIDENCE-BURGLARY ALARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6031,53,ROBERRY/HOLD UP ALARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6032,70,RESIDENCE-BURGLARY ALARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
